In [8]:
import torch
import numpy as np
from PIL import Image
import math

# 1. Create simple 3D Gaussians

def create_gaussians(n=200):
    means = torch.randn(n, 3)                      # 3D points
    means[:, 2] += 4.0                             # move forward
    cov = torch.rand(n) * 0.05 + 0.02              # size
    colors = torch.rand(n, 3)                      # RGB
    return means, cov, colors



# 2. Project 3D → 2D

def project(means, fx=500, fy=500, cx=256, cy=256):
    x, y, z = means[:,0], means[:,1], means[:,2]
    u = fx * (x / z) + cx
    v = fy * (y / z) + cy
    return u, v, z

# 3. Gaussian Splatting Renderer (CPU)

def render_gaussians(means, cov, colors, H=512, W=512):
    img = torch.zeros(H, W, 3)
    u, v, z = project(means)

    sigma = 500 * cov / z       # world → pixel size

    for i in range(len(means)):
        ui, vi, zi = float(u[i]), float(v[i]), float(z[i])
        si = float(sigma[i])
        r, g, b = colors[i]

        if zi <= 0:
            continue

        # bounding box
        xmin = int(max(ui - 3*si, 0))
        xmax = int(min(ui + 3*si, W-1))
        ymin = int(max(vi - 3*si, 0))
        ymax = int(min(vi + 3*si, H-1))

        for px in range(xmin, xmax):
            for py in range(ymin, ymax):
                dx = (px - ui) / si
                dy = (py - vi) / si

                # USE math.exp (fix)
                w = math.exp(-(dx*dx + dy*dy))

                a = 0.05 * w   # opacity

                img[py, px] = img[py, px] * (1 - a) + torch.tensor([r, g, b]) * a

    return img


# 3D Gaussian Visualization ---
import numpy as np
import plotly.graph_objects as go

# Create 3D Gaussians on a sphere
def create_3d_gaussians(n=1500, radius=1.0, jitter=0.05):
    theta = np.random.uniform(0, 2*np.pi, n)
    phi = np.random.uniform(0, np.pi, n)

    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)

    # little noise
    x += np.random.randn(n) * jitter
    y += np.random.randn(n) * jitter
    z += np.random.randn(n) * jitter

    # Gaussian size
    sigma = np.random.uniform(0.03, 0.1, n)

    # Color based on height
    colors = (z - z.min()) / (z.max() - z.min())

    return x, y, z, sigma, colors

# Generate Gaussians
x, y, z, sigma, colors = create_3d_gaussians()

# Plot in 3D
fig = go.Figure(data=[
    go.Scatter3d(
        x=x, y=y, z=z,
        mode="markers",
        marker=dict(
            size=sigma * 60,
            color=colors,
            colorscale="turbo",
            opacity=0.85
        )
    )
])

fig.update_layout(
    title="3D Gaussian Splatting Visualization",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    ),
    width=900, height=700
)

fig.show()

# 4. MAIN

means, cov, colors = create_gaussians(300)
img = render_gaussians(means, cov, colors)

img = (img.numpy() * 255).astype(np.uint8)
Image.fromarray(img).save("simple_3d_gaussian_splatting.png")

print("Saved: simple_3d_gaussian_splatting.png")


Saved: simple_3d_gaussian_splatting.png
